In [1]:
import subprocess
import os
import datetime
import ipywidgets as widgets
import pandas as pd
import numpy as np
import folium
from folium import plugins
import branca.colormap as cm
import matplotlib.pyplot as plt
import mplleaflet
#from matplotlib.patches import Ellipse
import math

In [2]:
from getDisplacementNGL import getDisplacement
from gps_interpolation import interpolate, create_grid, reshape_and_create_df

In [3]:
# create map
def create_map(df, scale, lon_col='Lon', lat_col='Lat', data_col='Delta V', site_col='Site'):
    '''
    Parameters
    ----------
    df :
        Pandas dataframe with column of lat, long, and a data column
    lon_col:
        Name of column with longitude coords. Default value of 'Lon'
    lat_col:
        Name of column with lattitude coords. Default value of 'Lat'
    data_col:
        Name of column with values to be colormapped. Default is 'Delta N'
    site_col:
        Name of column with GPS site names. 
    -------
    Returns:
        Folium map object
    '''
    start_cords = (df[lat_col].median(),
                   df[lon_col].median())
    my_map = folium.Map(start_cords,
                        tiles=None,zoom_start=8, width=750, height=465,contol_scale=True)
    plugins.Fullscreen(position='topleft').add_to(my_map)
    # load basemap with name
    folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}',
                        attr='Tiles &copy; Esri', name='World_Topo_Map').add_to(my_map)

    
    # making MPL seismic colormap in Branca
    colors_list = ['blue', 'white', 'red']
    minimum = df[data_col].min()
    maximum = df[data_col].max()
    colormap = cm.LinearColormap(
        colors=colors_list, vmin=minimum, vmax=maximum)
    stations = folium.FeatureGroup(
        name='Stations')
    vertical = folium.FeatureGroup(
        name='Vertical Displacement')
    horizontal = folium.FeatureGroup(
        name='Horizontal Displacement')
    # displace scale, mm/yer/deg
    
    icon_url = "https://maps.google.com/mapfiles/kml/paddle/grn-blank.png"
    html = '<a target="_blank" href="https://sideshow.jpl.nasa.gov/post/links/{site}.html"><img src="https://sideshow.jpl.nasa.gov/post/plots/{site}.jpg" width="300" height="300"></a>'
    for cord, disp, site in zip(zip(df[lat_col], df[lon_col]), df[data_col], df[site_col]):
        folium.Circle(
            location=cord,
            radius=abs(disp)*scale,
            fill=True,
            color=colormap(disp),
            popup=f'Vertical Displacement:{"{:10.4f}".format(disp)} mm,   Site:{site}',
            stroke=True,
            weight=1,
            fill_opacity=.8
        ).add_to(vertical)
        icon = folium.features.CustomIcon(icon_url,icon_size=(16, 16),icon_anchor=(8,16))
        #iframe = folium.element.IFrame(html=html, width=500, height=300)
        s_html = html.format(site=site)
        popup = folium.Popup(s_html, max_width=2650)

        folium.Marker(
            location=cord,
            icon=icon,
            popup=popup,
        ).add_to(stations)
    
    horizontal = folium.FeatureGroup(
        name='Horizontal Displacement')
    for lat,lon,vlon,vlat in zip(df[lat_col], df[lon_col],df['Delta E'],df['Delta N']):
        linedis = [[lat,lon],[lat+vlat/scale,lon+vlon/scale/math.cos(lat*math.pi/180.)]]
        folium.PolyLine(
            locations=linedis,
            color='red',
            weight=2
        ).add_to(horizontal)
                   
    errorlayer = folium.FeatureGroup(
        name='Error eclipse')
    for lat,lon,vlon,vlat,slon,slat in zip(df[lat_col], df[lon_col],df['Delta E'],df['Delta N'],df['Sigma E'],df['Sigma N']):
        theta = 0
        coords = []
        for k in range(0,31):
            angle = k/30*2*math.pi
            elon = slon*math.cos(angle)*math.cos(theta)-slat*math.sin(angle)*math.sin(theta)
            elat = slon*math.cos(angle)*math.sin(theta)+slat*math.sin(angle)*math.cos(theta)
            elon = (elon+vlon)/scale/math.cos(lat*math.pi/180.)
            elat = (elat+vlat)/scale
            coords.append([lat+elat,lon+elon])
        folium.Polygon(
            locations=coords,
            fill=False,
            color="black",
            weight=2 
        ).add_to(errorlayer)
    my_map.add_child(stations)
    my_map.add_child(vertical)
    my_map.add_child(horizontal)
    my_map.add_child(errorlayer)
    my_map.add_child(colormap)
#     folium.raster_layers.ImageOverlay('error_elippse.png', [[df[lat_col].min(
#     ), df[lon_col].min()], [df[lat_col].max(), df[lon_col].max()]], name='Error Ellipse').add_to(my_map)
    my_map.add_child(folium.LayerControl(postion='bottomleft'))
    
    return my_map

In [4]:
# run iterpolation

def to_los_disp(ux, uy, uv, azimuth=-5, elevation=60):
    g = [math.sin(azimuth)*math.cos(elevation), math.cos(azimuth)
         * math.cos(elevation), math.sin(elevation)]
    losd = (g[0]*ux + g[1]*uy + g[2]*uv)/5.0
    return losd

def runinterpolation(data):
    # parameters for interpolation
    # global values
    global gps_df
    global deltas
    global interpolated_values
    gps_df = data
    deltas = gps_df[['Delta E', 'Delta N', 'Delta V']]

    interpolated_values = interpolate(
        gps_df['Lon'],
        gps_df['Lat'],
        grid_spacing=grid_space.value,
        model=interpolation_type.value,
        **deltas)
    losd = to_los_disp(interpolated_values['Delta E'], interpolated_values['Delta N'],
                       interpolated_values['Delta V'], elevation=elevation.value, azimuth=azimuth.value)
    interpolated_values['LOS Displacement'] = losd


In [5]:
# run model
def runmodel(b):
    # required
    parameters = {"lat":lat.value,"lon":lon.value,"width":width.value,"height":height.value,"epoch1":t1.value.strftime("%Y-%m-%d"),"epoch2":t2.value.strftime("%Y-%m-%d"),"output":file_name.value,'analysisCenter':analysisCenter.value, "scale":vscale.value}
    # optional
    if (len(ref_site.value)==4):
        parameters['ref'] = ref_site.value
    else:
        parameters['ref'] = None 
    parameters['ct'] = None 
    parameters['pt'] = None
    parameters['eon'] = True
    parameters['mon'] = False
    parameters['dwin1'] = None
    parameters['dwin2'] = None
    parameters['vabs'] = False
    infolb.value = "run getDisplacement ..."
    data_table = getDisplacement(parameters)
    datadf = pd.DataFrame(data_table, columns=["Site","Lon","Lat","Delta E","Delta N","Delta V","Sigma E","Sigma N","Sigma V"])
    
    # for interpolation 
    runinterpolation(datadf)
    
    infolb.value = "generate map ..."
    my_map = create_map(datadf,float(parameters['scale']))
    map_output.clear_output()
    with map_output:
        display(my_map)
    infolb.value = "done!"

In [6]:
# input parameters for interpolation
interp_button = widgets.Button(description='Update interpolation')
grid_space = widgets.FloatSlider(value=0.018,
                                 min=0.005,
                                 max=.1,
                                 step=.0005,
                                 description='Grid Spacing: ',
                                 disabled=False,
                                 continous_update=True,
                                 readout=True,
                                 readout_format='.4f',
                                 default=.018)
interpolations = ['linear', 'gaussian', 'power',
                  'exponential', 'hole-effect', 'spherical']
interpolation_type = widgets.RadioButtons(options=interpolations,
                                          description="Interpolation",
                                          default='linear')
azimuth = widgets.IntSlider(value=-5,
                            min=-180,
                            max=180,
                            step=5,
                            description='Azimuth',
                            disabled=False,
                            readout=True,
                            default=-5)
elevation = widgets.IntSlider(value=60,
                              min=0,
                              max=180,
                              step=1,
                              description='Elevation Angle',
                              disabled=False,
                              readout=True,
                              default=60)
interpolation_tab = widgets.VBox([grid_space, interpolation_type,
                      azimuth, elevation,interp_button ])

In [7]:
# input parameters for model
style = {'description_width': '150px'}
lat = widgets.FloatText(
    value=33.0,
    description='Lat',
    step=0.1,
    style=style
)
lon = widgets.FloatText(
    value=-115.0,
    description='Lon',
    step=0.1,
    style=style
)
width = widgets.FloatText(
    value=1,
    description='Width (degree)',
    step=0.1,
    style=style
)
height = widgets.FloatText(
    value=1,
    description='Height (degree)',
    step=0.1,
    style=style
)
analysisCenter = widgets.RadioButtons( 
    options=['JPL', 'NGL'], description ='Data Source',disabled=False,style=style)

vscale = widgets.FloatText(
    value=320,
    description='Scale mm/yr/dgree',
    step=10,
    style=style
)
ref_site= widgets.Text(description='Ref. Site', value='',style=style)
t1 = widgets.DatePicker(description='Time 1',value = datetime.date(2010,4,7),style=style)
t2 = widgets.DatePicker(description='Time 2',value = datetime.date(2010,4,9),style=style)
file_name = widgets.Text(description='Output name', value='displacement',style=style)
b = widgets.Button(description='Run')
infolb = widgets.Label()
b.on_click(runmodel)
all_widgets = [lat, lon, width, height, ref_site,t1, t2, vscale, analysisCenter, file_name,b, infolb]
model_tab = widgets.VBox(all_widgets)
tab_nest = widgets.Tab()
tab_nest.children = [model_tab, interpolation_tab]
tab_nest.set_title(0,"Model")
tab_nest.set_title(1,"Interpolation")
#tab_nest
map_output = widgets.Output()
vbox1 = widgets.VBox([tab_nest,map_output])
vbox1

In [10]:
gps_df

,Site,Lon,Lat,Delta E,Delta N,Delta V,Sigma E,Sigma N,Sigma V
0,GMPK,-114.827347,33.051084,1.769640,-1.970650,-0.684816,0.313947,0.365123,1.364453
1,IID2,-115.031804,32.706168,7.359603,-4.292745,-0.992774,0.297097,0.350955,1.247817
2,P499,-115.487914,32.979606,1.931054,-9.400469,0.668483,0.291891,0.342312,1.219653
3,P500,-115.299933,32.690046,12.283891,-12.540173,0.438303,0.280181,0.336806,1.140766
4,P501,-115.397914,32.875756,3.109161,-11.023336,0.828573,0.287773,0.348138,1.191560
5,P502,-115.421913,32.982450,1.870937,-8.353048,1.136570,0.290019,0.352150,1.200679
6,P508,-115.428707,33.247780,0.354677,-4.670191,0.651837,0.288847,0.345983,1.187426
7,P509,-115.293919,32.890662,3.064828,-7.699326,0.606318,0.277898,0.334891,1.126954
8,P510,-115.343334,33.143574,0.552432,-5.217309,-0.144948,0.290733,0.348747,1.245531
